# Протоколы аутентификации

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

## Протокол PAP
![PAP Protocol](./images/PAP_1.png)

-----

In [3]:
# Использую хэш-функцию ГОСТ 34.11-2012 "Стрибог"
from pygost.gost34112012256 import GOST34112012256

# Эта либа тоже понадобится
import binascii

# Для генерации случайных чисел (КГПСЧ)
from Crypto.Random import get_random_bytes

----

In [4]:
def streebog_hash(*args) -> bytes:
    """
    Расчет хэша для нескольких аргументов (по типу конкатенации)
    """
    
    hasher = GOST34112012256()
    
    for arg in args:
        hasher.update(arg)
        
    return hasher.digest()

def bytes_as_hex(b: bytes) -> str:
    """
    Перевод бинарных данных в hex-строчку
    """
    
    return binascii.hexlify(b).decode()

------

In [5]:
class PAPServer(object):
    def __init__(self):
        self._db = {}
        
    def register_user(self, login: str, password: str):
        trace('[PAP Server]', f'Attempt to register user with credentials {login}:{password}')
        
        if login in self._db:
            trace('[PAP Server]', 'User already exists')
            raise ValueError(f'User {login} is already registered')
            
        #
        # Рассчитаю хэш пароля и запишу в базу
        #
        
        self._db[login] = streebog_hash(password.encode())
        trace('[PAP Server]', f'User successfully registered, password hash: {bytes_as_hex(self._db[login])}')
        
    def login(self, login: str, password: str) -> bool:
        trace('[PAP Server]', f'Attempt to login with credentials {login}:{password}')
        
        if login not in self._db:
            trace('[PAP Server]', "User doesn't exist")
            return False
        
        #
        # Пользователь есть, рассчитаю хэш пароля и сравню с имеющимся
        #
        
        pass_hash = streebog_hash(password.encode())
        real_hash = self._db[login]
        
        trace('[PAP Server]', f'Password hash: {bytes_as_hex(pass_hash)}')
        trace('[PAP Server]', f'Stored hash:   {bytes_as_hex(real_hash)}')
        
        return pass_hash == real_hash

In [6]:
class PAPClient(object):
    def __init__(self, login, password):
        self._login  = login
        self._passwd = password
        
        trace('[PAP Client]', f'User with credentials {self._login}:{self._passwd} created')
        
    def register(self, srv: PAPServer):
        try:
            srv.register_user(self._login, self._passwd)
        except Exception as e:
            trace(e)
    
    def login(self, srv: PAPServer):
        if srv.login(self._login, self._passwd):
            print('[PAP Client]', f'User {self._login} logged in successfully')
        else:
            print('[PAP Client]', f'Wrong username or password')

----

In [7]:
# Создаю сервер
server = PAPServer()

In [8]:
# Теперь создаю двух юзеров: 
# - Алису - валидного пользователя
# - Еву - пользователя, пытающегося представиться Алисой
real_alice = PAPClient('Alice', 'Trust_m3_0r_n0t')
eve = PAPClient('Alice', 'try_t0_gu3$$')

[TRACE] [PAP Client] User with credentials Alice:Trust_m3_0r_n0t created
[TRACE] [PAP Client] User with credentials Alice:try_t0_gu3$$ created


In [9]:
# Регистрирую валидного пользователя
real_alice.register(server)

[TRACE] [PAP Server] Attempt to register user with credentials Alice:Trust_m3_0r_n0t
[TRACE] [PAP Server] User successfully registered, password hash: bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8


In [10]:
# Захожу под кредами валидного пользователя
real_alice.login(server)

[TRACE] [PAP Server] Attempt to login with credentials Alice:Trust_m3_0r_n0t
[TRACE] [PAP Server] Password hash: bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8
[TRACE] [PAP Server] Stored hash:   bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8
[PAP Client] User Alice logged in successfully


In [11]:
# Пытаюсь зайти под неправильными кредами
eve.login(server)

[TRACE] [PAP Server] Attempt to login with credentials Alice:try_t0_gu3$$
[TRACE] [PAP Server] Password hash: 034eda071f984d17f28f9ae214c5d9b9a9fc0e112df98d0e4b5b2eb3d701b314
[TRACE] [PAP Server] Stored hash:   bdb147b4b8ae6f408dd51cdb17d6300f3e475b504129ba01e5715899b60366b8
[PAP Client] Wrong username or password


-----

## Протокол CHAP
![CHAP Protocol](./images/CHAP_1.png)

In [12]:
class CHAPServer(object):
    def __init__(self):
        self._db = {}
        
    def register_user(self, login, password):
        trace('[CHAP Server]', f'Attempt to register user with credentials {login}:{password}')
        
        if login in self._db:
            trace('[CHAP Server]', 'User already exists')
            raise ValueError(f'User {login} is already registered')
            
        #
        # Запишу в базу
        #
        
        self._db[login] = [password.encode(), None]
        trace('[CHAP Server]', f'User successfully registered')
    
    def login(self, login, response):
        trace('[CHAP Server]', f'''Attempt to login with: {login = }, 
        response = {bytes_as_hex(response)}''')
        
        if login not in self._db:
            trace('[CHAP Server]', "User doesn't exist")
            return False
        
        #
        # Пользователь есть, достану челлендж и обнулю
        #
        
        challenge = self._db[login][1]
        self._db[login][1] = None
        
        if challenge is None:
            trace('[CHAP Server]', f'No challenge was generated for {login = }')
            return False
        
        #
        # Рассчитаю хэш и сравню с респонсом
        #
        
        challenge_hash = streebog_hash(challenge, self._db[login][0])
        
        trace('[CHAP Server]', f'Calculated hash: {bytes_as_hex(challenge_hash)}')
        trace('[CHAP Server]', f'Received hash:   {bytes_as_hex(response)}')
        
        return challenge_hash == response
    
    def generate_challenge(self, login):
        trace('[CHAP Server]', f'Attempt to generate challege for: {login = }')
        
        if login not in self._db:
            trace('[CHAP Server]', "User doesn't exist")
            return None
        
        challenge = get_random_bytes(16)
        self._db[login][1] = challenge
        
        trace('[CHAP Server]', f'Generated challenge = {bytes_as_hex(challenge)}')
        return challenge

In [13]:
class CHAPClient(object):
    def __init__(self, login, password):
        self._login  = login
        self._passwd = password
        
        trace('[CHAP Client]', f'User with credentials {self._login}:{self._passwd} created')
        
    def register(self, srv: CHAPServer):
        try:
            srv.register_user(self._login, self._passwd)
        except Exception as e:
            trace(e)
    
    def login(self, srv: CHAPServer):
        challenge = srv.generate_challenge(self._login)
        if challenge is None:
            print('[CHAP Client]', f'Wrong username or password')
            return
        
        response = streebog_hash(challenge, self._passwd.encode())
        if srv.login(self._login, response):
            print('[CHAP Client]', f'User {self._login} logged in successfully')
        else:
            print('[CHAP Client]', f'Wrong username or password')

----

In [14]:
# Создаю сервер
server = CHAPServer()

In [15]:
# Теперь создаю двух юзеров: 
# - Алису - валидного пользователя
# - Еву - пользователя, пытающегося представиться Алисой
real_alice = CHAPClient('Alice', 'Trust_m3_0r_n0t')
eve = CHAPClient('Alice', 'try_t0_gu3$$')

[TRACE] [CHAP Client] User with credentials Alice:Trust_m3_0r_n0t created
[TRACE] [CHAP Client] User with credentials Alice:try_t0_gu3$$ created


In [16]:
# Регистрирую валидного пользователя
real_alice.register(server)

[TRACE] [CHAP Server] Attempt to register user with credentials Alice:Trust_m3_0r_n0t
[TRACE] [CHAP Server] User successfully registered


In [17]:
# Захожу под кредами валидного пользователя
real_alice.login(server)

[TRACE] [CHAP Server] Attempt to generate challege for: login = 'Alice'
[TRACE] [CHAP Server] Generated challenge = fdafb43f36922d02d56ffd3de292de49
[TRACE] [CHAP Server] Attempt to login with: login = 'Alice', 
        response = 9de261a764d7c1b1eb167f1d8cc679f07c2635dcb4d66765217138a1c188056b
[TRACE] [CHAP Server] Calculated hash: 9de261a764d7c1b1eb167f1d8cc679f07c2635dcb4d66765217138a1c188056b
[TRACE] [CHAP Server] Received hash:   9de261a764d7c1b1eb167f1d8cc679f07c2635dcb4d66765217138a1c188056b
[CHAP Client] User Alice logged in successfully


In [18]:
# Пытаюсь зайти под неправильными кредами
eve.login(server)

[TRACE] [CHAP Server] Attempt to generate challege for: login = 'Alice'
[TRACE] [CHAP Server] Generated challenge = c81d8164fd83f74de900bc1e36029f6f
[TRACE] [CHAP Server] Attempt to login with: login = 'Alice', 
        response = eaaa3bc5bd0e0df8c4caf1e4bb86acf3ec091ffcf7db1d081dce93d309f94ca2
[TRACE] [CHAP Server] Calculated hash: a0adf06cbe7a1ea36f2c483eb24c8ac35f8ea72fcb2d73522980a41f23f74e6a
[TRACE] [CHAP Server] Received hash:   eaaa3bc5bd0e0df8c4caf1e4bb86acf3ec091ffcf7db1d081dce93d309f94ca2
[CHAP Client] Wrong username or password


----

## Двустронний протокол CHAP

In [19]:
class ModifiedCHAPServer(object):
    def __init__(self):
        self._db = {}
        self._server_login = 'Alice'
        self._server_pass  = '1qazXSW@'
        
    def register_user(self, login, password):
        trace('[Modified CHAP Server]', f'Attempt to register user with credentials {login}:{password}')
        
        if login in self._db:
            trace('[Modified CHAP Server]', 'User already exists')
            raise ValueError(f'User {login} is already registered')
            
        #
        # Запишу в базу
        #
        
        self._db[login] = [password.encode(), None]
        
        trace('[Modified CHAP Server]', f'User successfully registered')
        
        return self._server_login, self._server_pass
        
    def login(self, login, response, client_challenge):
        trace(f'''[Modified CHAP Server]', f'Attempt to login with: {login = },
        response = {bytes_as_hex(response)}, 
        client_challenge = {bytes_as_hex(client_challenge)}''')
        
        if login not in self._db:
            trace('[Modified CHAP Server]', "User doesn't exist")
            return None
        
        #
        # Пользователь есть, достану челлендж и обнулю
        #
        
        challenge = self._db[login][1]
        self._db[login][1] = None
        
        if challenge is None:
            trace('[Modified CHAP Server]', f'No challenge was generated for {login = }')
            return None
        
        #
        # Рассчитаю хэш и сравню с респонсом
        #
        
        challenge_hash = streebog_hash(challenge, self._db[login][0])
        
        trace('[Modified CHAP Server]', f'Calculated hash: {bytes_as_hex(challenge_hash)}')
        trace('[Modified CHAP Server]', f'Received hash:   {bytes_as_hex(response)}')
        
        if challenge_hash != response:
            return None
        
        #
        # Теперь считаю хэш для челленджа клиента и возвращаю его
        #
        
        client_challenge_hash = streebog_hash(client_challenge, self._server_pass.encode())
        return self._server_login, client_challenge_hash
    
    def generate_challenge(self, login):
        trace('[Modified CHAP Server]', f'Attempt to generate challege for: {login = }')
        
        if login not in self._db:
            trace('[Modified CHAP Server]', "User doesn't exist")
            return None
        
        challenge = get_random_bytes(16)
        self._db[login][1] = challenge
        
        trace('[Modified CHAP Server]', f'Generated challenge = {bytes_as_hex(challenge)}')
        
        return challenge

In [20]:
class ModifiedCHAPClient(object):
    def __init__(self, login, password):
        self._login  = login
        self._passwd = password
        
        self._servers_db = {}
        
        trace('[Modified CHAP Client]', f'User with credentials {self._login}:{self._passwd} created')
        
    def register(self, srv: CHAPServer):
        try:
            login, password = srv.register_user(self._login, self._passwd)
            self._servers_db[login] = password
        except Exception as e:
            trace(e)
    
    def login(self, srv: CHAPServer):
        srv_challenge = srv.generate_challenge(self._login)
        if srv_challenge is None:
            print('[Modified CHAP Client]', f'Wrong username or password')
            return
        
        response  = streebog_hash(srv_challenge, self._passwd.encode())
        challenge = self.generate_challenge()
        srv_data  = srv.login(self._login, response, challenge)
        
        if srv_data is not None:
            srv_login, srv_response = srv_data
            
            if srv_login not in self._servers_db:
                print('[Modified CHAP Client]', f'Server {srv_login} not registered')
                return
            
            srv_challenge_hash = streebog_hash(challenge, self._servers_db[srv_login].encode())
            
            trace('[Modified CHAP Client]', f'Calculated hash: {bytes_as_hex(srv_challenge_hash)}')
            trace('[Modified CHAP Client]', f'Received hash:   {bytes_as_hex(srv_response)}')
            
            if srv_challenge_hash == srv_response:
                print('[Modified CHAP Client]', f'User {self._login} logged in successfully')
            else:
                print('[Modified CHAP Client]', f'Server {srv_login} is not authenticated')
        
        print('[Modified CHAP Client]', f'Wrong username or password')
            
    @staticmethod
    def generate_challenge():
        return get_random_bytes(16)

------

In [21]:
# Создаю сервер
server = ModifiedCHAPServer()

In [22]:
# Теперь создаю двух юзеров: 
# - Алису - валидного пользователя
# - Еву - пользователя, пытающегося представиться Алисой
real_alice = ModifiedCHAPClient('Alice', 'Trust_m3_0r_n0t')
eve = ModifiedCHAPClient('Alice', 'try_t0_gu3$$')

[TRACE] [Modified CHAP Client] User with credentials Alice:Trust_m3_0r_n0t created
[TRACE] [Modified CHAP Client] User with credentials Alice:try_t0_gu3$$ created


In [23]:
# Регистрирую валидного пользователя
real_alice.register(server)

[TRACE] [Modified CHAP Server] Attempt to register user with credentials Alice:Trust_m3_0r_n0t
[TRACE] [Modified CHAP Server] User successfully registered


In [24]:
# Захожу под кредами валидного пользователя
real_alice.login(server)

[TRACE] [Modified CHAP Server] Attempt to generate challege for: login = 'Alice'
[TRACE] [Modified CHAP Server] Generated challenge = 973a821615abbfa41e864d745232f78a
[TRACE] [Modified CHAP Server]', f'Attempt to login with: login = 'Alice',
        response = 62521dc017158da562173536ab25894cc72264ff1a558910301157e8dbaddc52, 
        client_challenge = 04686ed4374881dd18d3566de8756df5
[TRACE] [Modified CHAP Server] Calculated hash: 62521dc017158da562173536ab25894cc72264ff1a558910301157e8dbaddc52
[TRACE] [Modified CHAP Server] Received hash:   62521dc017158da562173536ab25894cc72264ff1a558910301157e8dbaddc52
[TRACE] [Modified CHAP Client] Calculated hash: a3985fb7aa4e1370a99b87f311321287541a4a71bf4e2c093cdd36aa2a37bd63
[TRACE] [Modified CHAP Client] Received hash:   a3985fb7aa4e1370a99b87f311321287541a4a71bf4e2c093cdd36aa2a37bd63
[Modified CHAP Client] User Alice logged in successfully
[Modified CHAP Client] Wrong username or password


In [25]:
# Пытаюсь зайти под неправильными кредами
eve.login(server)

[TRACE] [Modified CHAP Server] Attempt to generate challege for: login = 'Alice'
[TRACE] [Modified CHAP Server] Generated challenge = 4115e74d804b8139481a7a83811cf192
[TRACE] [Modified CHAP Server]', f'Attempt to login with: login = 'Alice',
        response = f145f180d896afa3f5df022d1536e2261453ebaa9afea881a0b647791c3ce70f, 
        client_challenge = 03b28013eb85d5b6861392f02bd41995
[TRACE] [Modified CHAP Server] Calculated hash: 7142792746178b32fb7bfad475563ecbc10890ee2f4ff9fe15a1ab9cdced4c81
[TRACE] [Modified CHAP Server] Received hash:   f145f180d896afa3f5df022d1536e2261453ebaa9afea881a0b647791c3ce70f
[Modified CHAP Client] Wrong username or password


----

## Протокол S/KEY
![S/KEY Protocol](./images/SKEY_1.png)

In [26]:
# To be implemented